In [14]:

import pytorch_lightning as pl
import torch
from torch import nn
from torchinfo import summary

In [15]:

class Model(pl.LightningModule):
    
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 32)
        self.linear2 = nn.Linear(28*28, 32)
        self.linear3 = nn.Linear(32+32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.flatten(x)

        x1 = self.linear1(x)
        x1 = self.relu(x1) 
        
        x2 = self.linear2(x)
        x2 = self.relu(x2)

        x3 = torch.cat([x1, x2], dim=1)
        y = self.linear3(x3) 
        
        return(y)


In [17]:
model = Model()
summary(model, input_size = (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Linear: 1-2                            [1, 32]                   25,120
├─ReLU: 1-3                              [1, 32]                   --
├─Linear: 1-4                            [1, 32]                   25,120
├─ReLU: 1-5                              [1, 32]                   --
├─Linear: 1-6                            [1, 10]                   650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.05
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.20
Estimated Total Size (MB): 0.21


----

하나의 모델이 다른 모델을 하위에 가지고 있는 형태

In [21]:
loss_ftn = nn.CrossEntropyLoss()

class MyModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.layers = Model()  ## 1개의 모델을 통으로 가지고 있는 형태 (상속이랑은 미묘하게 다름 -> 여러개 모델을 동시에 거느리는걸 생각해보자)

    
    def forward(self, x):
        out = self.layers(x)
        return(out)


    def predict_step(self, x, batch_idx):  ## pred 에서는 x 만 들어오기 때문에 batch 대신 x 라고 표시
        y_pred = self(x) # 여기까진  logit 
        y_prob = nn.Softmax(y_pred)  # 확률로 변환 
        return(y_prob)

     
    def traning_step(self, batch, batch_idx): ## 학습시에는 (x, y) 쌍이 들어오므로 batch 라고 표현
        
        




